# Generation of Javascript file with DIRAC output

### -- Michel Villanueva

The Javascript generator code is called

In [1]:
from js_gen import *

The file "sites.csv" contains the coordinates of the sites

In [2]:
data_coordinates = read_data("input/sites.csv", split = ',')
for site in data_coordinates:
    print site

['Site', 'CoordinateX', 'CoordinateY\r']
['Adelaide', '138.57121', '-34.910836\r']
['SIGNET', '14.48767', '46.04217\r']
['AWS_Tokyo', '139.763', '35.784498\r']
['CC1_Krakow', '19.890541', '50.089557\r']
['BINP', '83.11', '54.85\r']
['Beihang', '116.3408333', '39.98\r']
['CINVESTAV', '-99.128603', '19.510869\r']
['DESY', '9.8772', '53.5772\r']
['Hokudai', '141.345669', '43.073511\r']
['Nara-WU', '135.828454', '34.68787\r']
['Niigata', '138.941511', '37.867478\r']
['Osaka-CU', '135.5075357', '34.5927358\r']
['PNNL-CASCADE', '-119.307833', '46.347283\r']
['PNNL-PIC', '-119.307833', '46.347283\r']
['PNNL', '-119.307833', '46.347283\r']
['TIFR', '72.81', '19.91\r']
['TMU', '139.377199', '35.61656\r']
['DIRAC.Tokyo', '139.763', '35.714\r']
['UAS', '-107.381713', '24.826213\r']
['UVic', '-123.31', '48.47\r']
['DIRAC.Yonsei.kr', '126.938575', '37.565807\r']
['CESNET', '14.3886', '50.102\r']
['CNAF', '11.3417', '44.4948\r']
['CYFRONET', '19.90916', '50.068978\r']
['Cosenza', '16.181456', '39.33

We create an object who handles the generacion of the javascript code:

In [3]:
map = JSgen('web/b2gridmap.js')

where `web/b2gridmap.js` is the output of the javascript file. The input for now is the **`gb2_site_summary`** command. (The main goal is to obtain this information inside Dirac enviroment using the Hayasaka-san script, but now I'm working on OSX and I cannot install Dirac client to set up the enviroment)

In [4]:
with open('info/gb2_site_summary.txt', 'r') as f:
    print(f.read())

Site                      Done    Failed   Running   Stalled   Waiting
ARC.SIGNET.si           164981      1868       872         1         1
CLOUD.AWS_Sydney.au       1020      1006        12         0         2
CLOUD.AWS_Tokyo.jp         369       110         1         0         2
CLOUD.AWS_Virginia.u      4002      1470        93         0         5
CLOUD.CC1_Krakow.pl      45411       445       202         0         2
DIRAC.BINP.ru           152344      6303       549         0         2
DIRAC.Beihang.cn         10939      1710       100         0         0
DIRAC.CINVESTAV.mx       47371      1533        94         0         0
DIRAC.DESY.de                2         0         0         0         1
DIRAC.Hokudai.jp          9025        88        40         0         0
DIRAC.Nara-WU.jp         11140       257        20         0         0
DIRAC.Niigata.jp         10418      2122        44         0         2
DIRAC.Osaka-CU.jp        17108       126        35         0         0
DIRAC.

The function **`read_gb2_site_summary`** reads the output, returning a list.

In [5]:
ce_obj = read_gb2_site_summary("info/gb2_site_summary.txt")

Now **`ce_obj`** is a list of objects. Each element contains the information of one site. 

In [ ]:
class CE_site:
    def __init__(self, se_name = 'CE'):
        self.name = se_name
        self.jobs_done = 0
        self.jobs_failed = 0
        self.jobs_running = 0
        self.jobs_waiting = 0
        self.jobs_stalled = 0
        self.coordinates = [0,0]

    def print_info(self):
        print "Name: " + self.name
        print "Jobs_succeeded: " + str(self.jobs_done)
        print "Jobs_failed: " + str(self.jobs_failed)
        print "Coordinates: " + str(self.coordinates[0]) + "," + str(self.coordinates[1])

For example:

In [6]:
ce_obj[1].print_info()

Name: CLOUD.AWS_Sydney.au
Jobs_succeeded: 1020
Jobs_failed: 1006
Coordinates: 151.2143191,-33.8607819


So, we include the computing elements on the map

In [7]:
for ce in ce_obj:
    map.add_ce_site(ce)

The input for the storage elements information is the **`gb2_list_se`** command output.

With  **`read_gb2_list_se`**  we pull the computing element sites information. 

In [8]:
se_obj = read_gb2_list_se("info/gb2_list_se.txt")


Again, the function returns a list of objects, in which each object represents the information of one SE site. 

In [ ]:
class SE_site:
    def __init__(self, ce_name = 'SE'):
        self.read = ''
        self.write = ''
        self.path = ''
        self.name = ce_name
        self.endpoint = ['', '']
        self.coordinates = [0,0]

    def print_info(self):
        print "Name: " + self.name
        print "Path: " + self.path
        print "Endpoint: " + self.endpoint[0] + " : " + self.endpoint[1]
        print "Read/Write: " + self.read + "/" + self.write
        print "Coordinates: " + str(self.coordinates[0]) + "," + str(self.coordinates[1])

For example:

In [9]:
se_obj[0].print_info()

Name: Adelaide-DATA-SE
Path: /dpm/ersa.edu.au/home/belle/DATA
Endpoint: coepp-dpm-01.ersa.edu.au : 8446/srm/managerv2?SFN=
Read/Write: Active/Active
Coordinates: 138.57121,-34.910836


Now, we include all this information on the map

In [10]:
for se in se_obj:
    map.add_se_site(se)

The function **`pull_dashboard`** searches the endpoint of the storage elements. When it finds a coincidence with the dashboard matrix, draws a line in the map. The color depends of the efficiency.

In [ ]:
    def pull_dashboard(self, path, hours=720):
        se1 = SE_site()
        se2 = SE_site()
        js_data = urllib2.urlopen(path).read()
        dashboard = json.loads(js_data)
        # For now, we only need the data transfer info
        data_matrix = dashboard['transfers']['rows']
        # Search the pair of SE elements
        for cell in data_matrix:
            for se in self.se_active:
                if se.endpoint[0] == cell[0]:
                    se1 = se;
            for se in self.se_active:
                if se.endpoint[0] == cell[1]:
                    se2 = se;
            #We calculate the speed on kBs
            speed = cell[2]/float(hours * 60 * 60 * 1000)
            efficiency = cell[3] *100 / (cell[3] + cell[4])

            description_text = """<strong>Source = """ + se1.name + """</br>
            Destination = """ + se2.name + """</strong></br><hr>
            <font style="font-weight: bold">Connection info:</font> </br>
            <div style="padding-left: 5px;">Throughput: %0.1f kB/s </br>"""%speed +\
            """Efficiency: %0.0f"""%efficiency +  """% </br>
            Transfer Successes: """ + str(cell[3]) +  """ </br>
            Transfer Failures: """ + str(cell[4]) + """  </br>
            </div><br />"""
            if efficiency < 20:
                color = '#FF0000'
            elif efficiency >= 20 and efficiency < 60:
                color = '#FFFF00'
            elif efficiency >= 60 and efficiency < 80:
                color = '#0033FF'
            else:
                color = '#00CC00'
            self.draw_line(se1, se2, color)
            self.lines_description.append(description_text)

Dashboard can display the info on json format. So, we pull the transfer information directly from the web site. Notice that the last number is the time interval (720 minutes = 12 hours)

In [11]:
map.pull_dashboard('http://dashb-fts-transfers.cern.ch/' + \
                    'dashboard/request.py/transfer-matrix.json?' + \
                    'vo=belle&server=b2fts3.cc.kek.jp&src_grouping=host'+ \
                    '&dst_grouping=host&interval=720',720)

Now, with this information we init the map and close the javascript file

In [12]:
map.init_map()
map.close()

And [**this**](http://jaguar.fis.cinvestav.mx/testing/gridmap) is the result when we put the js file running on google maps.

In [13]:
from IPython.display import HTML
HTML("<iframe src='http://jaguar.fis.cinvestav.mx/testing/gridmap/' width = 800 height = 600></iframe>")